In [1]:
import keras
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Input
from keras.models import Sequential
import tensorflow as tf
import numpy as np
from pymongo import MongoClient
import pydicom as pyd
import matplotlib.pyplot as plt
import pandas as pd
from skimage.transform import resize
from skimage import measure
import random
from random import shuffle
from PIL import Image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers import BatchNormalization, Input, Conv2D, multiply, LocallyConnected2D, LeakyReLU, AvgPool2D
from keras.applications.densenet import DenseNet121
from keras import Model

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
client = MongoClient('localhost', 27017)
db = client['rsna-xray']
col = db['patient']

In [3]:
#Setting up training and testing data
normal = list(col.find({"class":"Normal"}))
lung_opacity = list(col.find({"class":"Lung Opacity"}))
not_normal = list(col.find({"class":"No Lung Opacity / Not Normal"}))

#Shuffling the documents
np.random.shuffle(normal)
np.random.shuffle(not_normal)
np.random.shuffle(lung_opacity)
#Getting first 4000 of the shuffled list
normal_train = normal[:4000]
opacity_train = lung_opacity[:4000]
not_normal_train = not_normal[:4000]

#getting 4001 to 6000 of each list
normal_test = normal[4000:6000]
opacity_test = lung_opacity[4000:6000]
not_normal_test = not_normal[4000:6000]

#Setting up training and testing list
train = normal_train + not_normal_train + opacity_train
np.random.shuffle(train)
test = normal_test + not_normal_test + opacity_test
np.random.shuffle(test)

In [4]:
#Funcion for batches
#Getting the next batch of X and y
def next_batch(batchsize, documents, size = (256, 256), shuffle = True,
               classes = ["No Lung Opacity / Not Normal", "Lung Opacity", "Normal"], stacked = False):
    """
    This function is meant to return the image array and the encoded labels for the 
    training of the neural network. 
    
    batchsize - int - how many instances you would want retreived
    
    documents - list - the list of documents you want the batch to come from
    
    size - tuple or list - the size of the output array want
    
    classes - Only change when not using the same three classes, keeps the labels the same
    """
    if shuffle:
        #randomly shuffles the documents
        idx = np.arange(0, len(documents))
        np.random.shuffle(idx)
        #Getting that number if indices
        idx = idx[:batchsize]
        #Creates a list for that batch
        batch = [documents[i] for i in idx]
        #A numpy matrix of the images
    else:
        batch = documents[:batchsize]
    
    #numpy array for the classes
    y = np.zeros([batchsize, 3])
    #Goes through the batch
    if stacked:
        X = np.zeros([batchsize, size[0], size[1], 3])
        for i, d in enumerate(batch):
            #reads the file
            f = pyd.read_file(d["path"])
            #Inserts the reshaped image in the data frame
            X[i] = resize(image=np.stack([f.pixel_array] * 3, axis=2), output_shape=size)
            #One hot encodes the class
            y[i, classes.index(d["class"])] = 1
    else:
        X = np.zeros([batchsize, size[0], size[1]])
        for i, d in enumerate(batch):
            #reads the file
            f = pyd.read_file(d["path"])
            #Inserts the reshaped image in the data frame
            X[i] = resize(image=f.pixel_array, output_shape=size)
            #One hot encodes the class
            y[i, classes.index(d["class"])] = 1        
    return X, y

In [5]:
#inception Base Model
densenet_pretrained = DenseNet121(include_top = False, weights="imagenet", input_shape = (256,256, 3), )
densenet_pretrained.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [6]:
#Shape out of the pretrained
pretrained_features = Input(densenet_pretrained.get_output_shape_at(0)[1:], name = 'feature_input')
#Number of kernels
norm_features = BatchNormalization()(pretrained_features)
global_pooling = GlobalAveragePooling2D()(norm_features)

pooling_dr = Dropout(0.25)(global_pooling)
dr_steps = Dropout(0.25)(Dense(128, activation = 'linear', use_bias=False)(pooling_dr))
dr_steps = BatchNormalization()(dr_steps)
dr_steps = LeakyReLU(0.1)(dr_steps)
out_layer = Dense(3, activation = 'softmax')(dr_steps)

out_model = Model(inputs = [pretrained_features], 
                   outputs = [out_layer], name = 'trained_model')

out_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_input (InputLayer)   (None, 8, 8, 1024)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 1024)        4096      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131072    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
__________

In [7]:
from keras.models import Sequential
from keras.optimizers import Adam

pneumonia_densenet = Sequential()
#Making inception not trainable
densenet_pretrained.trainable = False
#Addign the DenseNet
pneumonia_densenet.add(densenet_pretrained)
#Adding the fully connected layers
pneumonia_densenet.add(out_model)
#Configuring the optimization of the model
pneumonia_densenet.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy',
                           metrics = ['categorical_accuracy'])
pneumonia_densenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 8, 8, 1024)        7037504   
_________________________________________________________________
trained_model (Model)        (None, 3)                 136067    
Total params: 7,173,571
Trainable params: 133,763
Non-trainable params: 7,039,808
_________________________________________________________________


In [8]:
%%time
#training on the whole dataset
for _ in np.arange(100):
    print(_)
    batch_xs, batch_ys = next_batch(24, train, shuffle=True, stacked=True)
    history = pneumonia_densenet.fit(x=batch_xs, y=batch_ys)

0


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch 1/1
24/24 [==============================] - 10s 408ms/step - loss: 1.5111 - categorical_accuracy: 0.1250
1
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 1.2903 - categorical_accuracy: 0.4167
2
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 1.3032 - categorical_accuracy: 0.4167
3
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 1.0015 - categorical_accuracy: 0.4583
4
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 1.0582 - categorical_accuracy: 0.5417
5
Epoch 1/1
24/24 [==============================] - 5s 220ms/step - loss: 1.1048 - categorical_accuracy: 0.5417
6
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 1.1809 - categorical_accuracy: 0.3333
7
Epoch 1/1
24/24 [==============================] - 5s 215ms/step - loss: 1.0585 - categorical_accuracy: 0.6250
8
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 1.3481 - categorical_a

72
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.9514 - categorical_accuracy: 0.6250
73
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.9879 - categorical_accuracy: 0.6250
74
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 0.4824 - categorical_accuracy: 0.8750
75
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.8291 - categorical_accuracy: 0.6250
76
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.7216 - categorical_accuracy: 0.6667
77
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.7765 - categorical_accuracy: 0.6250
78
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 1.1947 - categorical_accuracy: 0.3750
79
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.7966 - categorical_accuracy: 0.6250
80
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 0.7809 - cat

In [21]:
batch_xs, batch_ys = next_batch(8, train, shuffle=True, stacked=True)
preds = pneumonia_densenet.predict_classes(batch_xs)
actual = np.argmax(batch_ys, axis=1)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [22]:
print(preds)
print(actual)

[0 2 2 0 2 2 2 0]
[1 2 2 0 2 0 0 1]


In [23]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true = actual, y_pred = preds)

0.5

In [24]:
%%time
#training on the whole dataset
for _ in np.arange(700):
    print(_)
    batch_xs, batch_ys = next_batch(24, train, shuffle=True, stacked=True)
    history = pneumonia_densenet.fit(x=batch_xs, y=batch_ys)

0


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch 1/1
24/24 [==============================] - 5s 218ms/step - loss: 0.6072 - categorical_accuracy: 0.6667
1
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.5723 - categorical_accuracy: 0.6667
2
Epoch 1/1
24/24 [==============================] - 5s 213ms/step - loss: 0.8682 - categorical_accuracy: 0.7083
3
Epoch 1/1
24/24 [==============================] - 5s 220ms/step - loss: 0.8716 - categorical_accuracy: 0.6250
4
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 1.0004 - categorical_accuracy: 0.6250
5
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.7841 - categorical_accuracy: 0.6667
6
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.7472 - categorical_accuracy: 0.6667
7
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.8020 - categorical_accuracy: 0.5833
8
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.9628 - categorical_ac

72
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.8539 - categorical_accuracy: 0.7083
73
Epoch 1/1
24/24 [==============================] - 5s 216ms/step - loss: 0.6056 - categorical_accuracy: 0.7083
74
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.8134 - categorical_accuracy: 0.5417
75
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.8286 - categorical_accuracy: 0.7083
76
Epoch 1/1
24/24 [==============================] - 5s 209ms/step - loss: 1.1219 - categorical_accuracy: 0.4583
77
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.9315 - categorical_accuracy: 0.5417
78
Epoch 1/1
24/24 [==============================] - 5s 215ms/step - loss: 0.9058 - categorical_accuracy: 0.5833
79
Epoch 1/1
24/24 [==============================] - 5s 219ms/step - loss: 0.9200 - categorical_accuracy: 0.5000
80
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.8987 - cat

144
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.6341 - categorical_accuracy: 0.7500
145
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.7407 - categorical_accuracy: 0.6667
146
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 1.0047 - categorical_accuracy: 0.5000
147
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.7970 - categorical_accuracy: 0.6667
148
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.9102 - categorical_accuracy: 0.6667
149
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.7885 - categorical_accuracy: 0.5833
150
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.8671 - categorical_accuracy: 0.5000
151
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.8700 - categorical_accuracy: 0.5000
152
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.8

...

360
Epoch 1/1
24/24 [==============================] - 5s 215ms/step - loss: 0.9824 - categorical_accuracy: 0.4167
361
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 1.1415 - categorical_accuracy: 0.3750
362
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 1.1090 - categorical_accuracy: 0.5417
363
Epoch 1/1
24/24 [==============================] - 5s 213ms/step - loss: 0.6958 - categorical_accuracy: 0.7083
364
Epoch 1/1
24/24 [==============================] - 5s 217ms/step - loss: 0.6426 - categorical_accuracy: 0.7917
365
Epoch 1/1
24/24 [==============================] - 5s 215ms/step - loss: 0.8459 - categorical_accuracy: 0.4583
366
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.7730 - categorical_accuracy: 0.6250
367
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.6997 - categorical_accuracy: 0.7500
368
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.8

432
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.5388 - categorical_accuracy: 0.7917
433
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.6196 - categorical_accuracy: 0.7083
434
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.6260 - categorical_accuracy: 0.7500
435
Epoch 1/1
24/24 [==============================] - 5s 209ms/step - loss: 0.6106 - categorical_accuracy: 0.6667
436
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.7648 - categorical_accuracy: 0.6667
437
Epoch 1/1
24/24 [==============================] - 5s 216ms/step - loss: 0.6207 - categorical_accuracy: 0.5833
438
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.6460 - categorical_accuracy: 0.7500
439
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.7931 - categorical_accuracy: 0.7917
440
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.7

504
Epoch 1/1
24/24 [==============================] - 5s 221ms/step - loss: 0.6082 - categorical_accuracy: 0.8333
505
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.6816 - categorical_accuracy: 0.5417
506
Epoch 1/1
24/24 [==============================] - 5s 216ms/step - loss: 0.6448 - categorical_accuracy: 0.6250
507
Epoch 1/1
24/24 [==============================] - 5s 213ms/step - loss: 0.6052 - categorical_accuracy: 0.7500
508
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.9210 - categorical_accuracy: 0.5417
509
Epoch 1/1
24/24 [==============================] - 5s 212ms/step - loss: 0.6990 - categorical_accuracy: 0.6667
510
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.7666 - categorical_accuracy: 0.5833
511
Epoch 1/1
24/24 [==============================] - 5s 213ms/step - loss: 0.5812 - categorical_accuracy: 0.8333
512
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.5

576
Epoch 1/1
24/24 [==============================] - 5s 209ms/step - loss: 0.7702 - categorical_accuracy: 0.8333
577
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.7525 - categorical_accuracy: 0.6250
578
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.6560 - categorical_accuracy: 0.7083
579
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.8420 - categorical_accuracy: 0.6667
580
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.7212 - categorical_accuracy: 0.6250
581
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 1.2217 - categorical_accuracy: 0.4583
582
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.7511 - categorical_accuracy: 0.7500
583
Epoch 1/1
24/24 [==============================] - 5s 217ms/step - loss: 0.8203 - categorical_accuracy: 0.5833
584
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.6

648
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.6387 - categorical_accuracy: 0.7500
649
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.8596 - categorical_accuracy: 0.6667
650
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.8933 - categorical_accuracy: 0.4583
651
Epoch 1/1
24/24 [==============================] - 5s 214ms/step - loss: 0.7766 - categorical_accuracy: 0.6250
652
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.7274 - categorical_accuracy: 0.7083
653
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 0.5230 - categorical_accuracy: 0.7917
654
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.9237 - categorical_accuracy: 0.5833
655
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.5750 - categorical_accuracy: 0.7500
656
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 0.6

In [36]:
batch_xs, batch_ys = next_batch(24, test, shuffle=True, stacked=True)
print(pneumonia_densenet.predict_classes(x=batch_xs))
print(np.argmax(batch_ys, axis = 1))

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[2 1 2 2 0 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2]
[0 1 1 0 1 2 2 0 2 1 1 1 2 0 1 0 2 2 0 2 2 2 2 1]


In [33]:
pneumonia_densenet.save("pneumonia_densenet_800.h5")
pneumonia_densenet.save_weights("pneumonia_densenet_weights_800.h5")
with open('pneumonia_classifier_architecture_densenet.json', 'w') as f:
    f.write(pneumonia_densenet.to_json())

In [37]:
%%time
#training on the whole dataset
for _ in np.arange(7500):
    print(_)
    batch_xs, batch_ys = next_batch(24, train, shuffle=True, stacked=True)
    history = pneumonia_densenet.fit(x=batch_xs, y=batch_ys)

0


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch 1/1
24/24 [==============================] - 5s 215ms/step - loss: 0.6566 - categorical_accuracy: 0.6667
1
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.8409 - categorical_accuracy: 0.6667
2
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.5981 - categorical_accuracy: 0.7500
3
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.6108 - categorical_accuracy: 0.5833
4
Epoch 1/1
24/24 [==============================] - 5s 210ms/step - loss: 0.5767 - categorical_accuracy: 0.8750
5
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.7156 - categorical_accuracy: 0.6667
6
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.8224 - categorical_accuracy: 0.5000
7
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.6967 - categorical_accuracy: 0.5000
8
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.8121 - categorical_ac

72
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 1.0033 - categorical_accuracy: 0.7083
73
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.6392 - categorical_accuracy: 0.7083
74
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.7803 - categorical_accuracy: 0.6667
75
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 0.9773 - categorical_accuracy: 0.5833
76
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.6179 - categorical_accuracy: 0.7500
77
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.9567 - categorical_accuracy: 0.7083
78
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.7250 - categorical_accuracy: 0.7500
79
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 0.5047 - categorical_accuracy: 0.8750
80
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.5368 - cat

144
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.9402 - categorical_accuracy: 0.5833
145
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.7125 - categorical_accuracy: 0.6667
146
Epoch 1/1
24/24 [==============================] - 5s 202ms/step - loss: 0.6981 - categorical_accuracy: 0.7083
147
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.9776 - categorical_accuracy: 0.6667
148
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.7573 - categorical_accuracy: 0.7500
149
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.8191 - categorical_accuracy: 0.6250
150
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.6971 - categorical_accuracy: 0.7083
151
Epoch 1/1
24/24 [==============================] - 5s 203ms/step - loss: 0.7167 - categorical_accuracy: 0.6250
152
Epoch 1/1
24/24 [==============================] - 5s 204ms/step - loss: 0.9

...

7185
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.5145 - categorical_accuracy: 0.7500
7186
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.5767 - categorical_accuracy: 0.7500
7187
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.7089 - categorical_accuracy: 0.7083
7188
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.6147 - categorical_accuracy: 0.7083
7189
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.5485 - categorical_accuracy: 0.7500
7190
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.6663 - categorical_accuracy: 0.6667
7191
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.8626 - categorical_accuracy: 0.5833
7192
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.4118 - categorical_accuracy: 0.7917
7193
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - 

7256
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.5898 - categorical_accuracy: 0.7917
7257
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.7135 - categorical_accuracy: 0.6667
7258
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.6581 - categorical_accuracy: 0.6667
7259
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.6986 - categorical_accuracy: 0.6250
7260
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.6080 - categorical_accuracy: 0.7083
7261
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.6007 - categorical_accuracy: 0.7917
7262
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.8976 - categorical_accuracy: 0.5000
7263
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.4237 - categorical_accuracy: 0.8333
7264
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - 

7327
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.4870 - categorical_accuracy: 0.7500
7328
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.8669 - categorical_accuracy: 0.5833
7329
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.5393 - categorical_accuracy: 0.8333
7330
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.7575 - categorical_accuracy: 0.6250
7331
Epoch 1/1
24/24 [==============================] - 5s 209ms/step - loss: 0.5543 - categorical_accuracy: 0.7917
7332
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.5177 - categorical_accuracy: 0.7500
7333
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.5495 - categorical_accuracy: 0.7083
7334
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.5350 - categorical_accuracy: 0.7083
7335
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - 

7398
Epoch 1/1
24/24 [==============================] - 5s 207ms/step - loss: 0.5961 - categorical_accuracy: 0.7083
7399
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 1.0229 - categorical_accuracy: 0.5000
7400
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.4674 - categorical_accuracy: 0.8750
7401
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.4743 - categorical_accuracy: 0.8333
7402
Epoch 1/1
24/24 [==============================] - 5s 208ms/step - loss: 0.7882 - categorical_accuracy: 0.6667
7403
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.4021 - categorical_accuracy: 0.9167
7404
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.8097 - categorical_accuracy: 0.6667
7405
Epoch 1/1
24/24 [==============================] - 5s 209ms/step - loss: 0.6420 - categorical_accuracy: 0.6250
7406
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - 

7469
Epoch 1/1
24/24 [==============================] - 5s 226ms/step - loss: 0.6094 - categorical_accuracy: 0.7083
7470
Epoch 1/1
24/24 [==============================] - 5s 225ms/step - loss: 0.4732 - categorical_accuracy: 0.8333
7471
Epoch 1/1
24/24 [==============================] - 5s 225ms/step - loss: 0.6803 - categorical_accuracy: 0.6250
7472
Epoch 1/1
24/24 [==============================] - 5s 222ms/step - loss: 0.7721 - categorical_accuracy: 0.6250
7473
Epoch 1/1
24/24 [==============================] - 5s 211ms/step - loss: 0.6271 - categorical_accuracy: 0.7083
7474
Epoch 1/1
24/24 [==============================] - 5s 205ms/step - loss: 0.6383 - categorical_accuracy: 0.7917
7475
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.4947 - categorical_accuracy: 0.8333
7476
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - loss: 0.5985 - categorical_accuracy: 0.8333
7477
Epoch 1/1
24/24 [==============================] - 5s 206ms/step - 

In [40]:
pneumonia_densenet.save("pneumonia_classifier_densenet.h5")
pneumonia_densenet.save_weights("pneumonia_classifier_densenet_weights.h5")
with open("pnuemonia_classifier_densenet_architecture.json", "w") as f:
    f.write(pneumonia_densenet.to_json())
    f.close()

In [42]:
#Predicting on the test set
#Empty list for prediction classes
prediction = []
#Class of actual values
actual = []
#Going through dataset
for i in np.arange(250):
    print(i)
    #starting index
    start = i * 24
    #End index
    end = (i + 1) * 24
    #Creating the batch dataset
    temp_data = test[start:end]
    #Cetting batch
    x_test, y_test = next_batch(24, temp_data, stacked=True, shuffle=False)
    #Putting the actual class in the actual list
    actual.extend(np.argmax(y_test, axis = 1))
    #Putting the predicted class in the predicted 
    prediction.extend(pneumonia_densenet.predict_classes(x_test))

0


/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [43]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(actual, prediction))
print(classification_report(actual, prediction, 
        target_names = ["No Lung Opacity / Not Normal", "Lung Opacity", "Normal"]))
test_accuracy = accuracy_score(y_pred=prediction, y_true=actual)
print("Accuracy on test set: {:.3f}".format(test_accuracy))

[[ 273  424 1303]
 [ 295  894  811]
 [  26   28 1946]]
                              precision    recall  f1-score   support

No Lung Opacity / Not Normal       0.46      0.14      0.21      2000
                Lung Opacity       0.66      0.45      0.53      2000
                      Normal       0.48      0.97      0.64      2000

                 avg / total       0.53      0.52      0.46      6000

Accuracy on test set: 0.519
